In [ ]:
# 关联客户信息和产品信息
from imp4nb import *

cnx = lite.connect('data\\quandan.db')
# desclitedb(cnx)

# cur = cnx.cursor()
# result = cur.execute('select * from xiaoshoumingxi')
# colnames = [tuple1[0] for tuple1 in result.description]
# print(colnames)
# print(len(result.fetchall()))
# print(result.fetchone()[0])

# df = pd.read_sql_query(
#     "select 日期,职员名称 as 业务主管,单据编号 as 销售单号,商品备注 as 订单单号,单据类型,摘要,xiaoshoumingxi.单位全名 as 客户,customer.往来单位编号 as 编码,"
#     "xiaoshoumingxi.商品全名 as 商品全名,数量,xiaoshoumingxi.单价,金额,substr(customer.往来单位编号,1,2) as 大区 ,"
#     "substr(customer.往来单位编号,1,3) as 小区 ,quyu.分部 as 分部,"
#     "substr(customer.往来单位编号,12,1) as 类型编码,leixing.类型 as 客户类型, product.品牌名称  as 品牌 ,product.品牌分类  as 品牌分类 from xiaoshoumingxi,"
#     "customer,product,quyu,leixing where (customer.往来单位 = xiaoshoumingxi.单位全名)  and (大区 = quyu.区域) "
#     "and (product.商品全名 = xiaoshoumingxi.商品全名) and (类型编码 = leixing.编码) order by 日期 desc", cnx,parse_dates=['日期'])
tm0 = time.clock()

df = pd.read_sql_query("select * from xiaoshoumingxi",cnx)
# descdb(df)
tm1 = time.clock()

df = pd.read_sql_query(
    "select 日期,职员名称 as 业务主管,单据编号 as 销售单号,商品备注 as 订单单号,单据类型,摘要,xiaoshoumingxi.单位全名 as 客户,customer.往来单位编号 as 编码,"
    "xiaoshoumingxi.商品全名 as 商品全名,数量,xiaoshoumingxi.单价,金额,substr(customer.往来单位编号,1,2) as 大区 ,"
    "substr(customer.往来单位编号,1,3) as 小区 ,quyu.分部 as 分部,"
    "substr(customer.往来单位编号,12,1) as 类型编码, product.品牌名称  as 品牌 ,product.品牌分类  as 品牌分类 from xiaoshoumingxi,"
    "customer,product,quyu where (customer.往来单位 = xiaoshoumingxi.单位全名)  and (大区 = quyu.区域) "
    "and (product.商品全名 = xiaoshoumingxi.商品全名) order by 日期 desc", cnx)
# descdb(df)
tm2 = time.clock()

df['日期'] = df['日期'].apply(lambda x: pd.to_datetime(x))
tm3 = time.clock()

cnm = lite.connect(":memory:")
df.to_sql(name='alldata', con=cnx, if_exists='replace',chunksize=50000)
cnm.close()
tm4 = time.clock()

df = pd.read_sql_query('select 日期,sum(金额) as 销售额 from alldata group by 日期',cnx)
# descdb(df)
tm5 = time.clock()

cnx.close()

print("起始时间：%f，从销售明细读取所有记录获取记录数：%f；"
      "关联客户、品牌和区域信息获得处理后的记录：%f；"
      "处理日期列：%f；"
      "写入内存数据库：%f；"
      "从存储数据库读取汇总销售数据：%f。" %(tm0,tm1-tm0,tm2-tm1,tm3-tm2,tm4-tm3,tm5-tm4))


In [3]:
from imp4nb import *

cnx = lite.connect('data\\quandan.db')

start = time.clock()
df = pd.read_sql_query('select * from xiaoshoumingxi',cnx)
finish1 = time.clock()
descdb(df)
finish2 = time.clock()

print("起始：%f，读取数据构建DataFrame：%f，描述DataFrame：%f。" %(start,finish1-start,finish2-finish1))

# dfcm = df.duplicated()
# print(list(dfcm[dfcm == True].index))
# df =df.drop_duplicates()
# descdb(df)

cnx.close()

1407895
         index                   日期                 单据编号   单据类型 职员名称    摘要  \
1407890   3167  2017-10-23 00:00:00  XT-2017-10-23-00074  销售退货单  朱经伦  None   
1407891   3168  2017-10-23 00:00:00  XT-2017-10-23-00074  销售退货单  朱经伦  None   
1407892   3169  2017-10-23 00:00:00  XT-2017-10-23-00074  销售退货单  朱经伦  None   
1407893   3170  2017-10-23 00:00:00  XT-2017-10-23-00074  销售退货单  朱经伦  None   
1407894   3171  2017-10-23 00:00:00  XT-2017-10-23-00074  销售退货单  朱经伦  None   

           备注  商品备注           商品编号            商品全名    单价 单位   数量     金额  \
1407890  None  None  6955504222608  抓鱼的猫盒（烧烤鱿鱼）13g  0.80  袋 -1.0  -0.80   
1407891  None  None  6952522000140      醉吃香山椒凤爪90g  3.20  袋 -3.0  -9.60   
1407892  None  None  6951957205816    劲仔盒鱼（酸菜味）12g  0.75  袋 -9.0  -6.75   
1407893  None  None  6951957202143   劲仔袋中袋鱼麻辣《50g》  5.00  袋 -4.0 -20.00   
1407894  None  None  6951957202136   劲仔袋中袋鱼香辣《50g》  5.00  袋 -3.0 -15.00   

                       单位全名      仓库全名  部门全名  
1407890  丰毅超市 13429886858 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1407895 entries, 0 to 1407894
Data columns (total 17 columns):
index    1407895 non-null int64
日期       1407895 non-null object
单据编号     1407748 non-null object
单据类型     1378797 non-null object
职员名称     1407895 non-null object
摘要       1377364 non-null object
备注       1121 non-null object
商品备注     1203585 non-null object
商品编号     1407895 non-null object
商品全名     1407895 non-null object
单价       1407021 non-null float64
单位       1406663 non-null object
数量       1407895 non-null float64
金额       1407021 non-null float64
单位全名     1407895 non-null object
仓库全名     1407726 non-null object
部门全名     1407719 non-null object
dtypes: float64(3), int64(1), object(13)
memory usage: 182.6+ MB
None
(1407895, 17)


              index            单价            数量            金额
count  1.407895e+06  1.407021e+06  1.407895e+06  1.407021e+06
mean   6.598455e+05  1.566409e+00  3.725578e+01  3.963854e+01
std    4.046364e+05  1.626280e+00  2.316851e+02  2.052122e+02
min    0.000000e+00  0.000000e+00 -4.860000e+03 -1.050000e+04
25%    3.067835e+05  7.500000e-01  1.000000e+01  1.500000e+01
50%    6.587570e+05  1.500000e+00  2.000000e+01  2.300000e+01
75%    1.010730e+06  1.800000e+00  3.000000e+01  3.800000e+01
max    1.362704e+06  8.800000e+01  8.000000e+04  5.571400e+04
起始：167.782680，读取数据构建DataFrame：65.849204，描述DataFrame：67.233739。
